In [1]:
import pyspark
from pyspark.sql.functions import *
sc = pyspark.SparkContext('local[*]')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2024-11-22T17:19:39,192 WARN [Thread-4] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import *

In [3]:
# import findspark
# findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Read Raw data

In [28]:
# 1
raw_olist_customers_dataset = spark.read.csv("/obligatorio/data/raw/olist_customers_dataset.csv", header=True, inferSchema=True)
# 2
raw_olist_geolocation_dataset = spark.read.csv("/obligatorio/data/raw/olist_geolocation_dataset.csv", header=True, inferSchema=True)
# 3
raw_olist_order_items_dataset = spark.read.csv("/obligatorio/data/raw/olist_order_items_dataset.csv", header=True, inferSchema=True)
# 4
raw_olist_order_payments_dataset = spark.read.csv("/obligatorio/data/raw/olist_order_payments_dataset.csv", header=True, inferSchema=True)
# 5
raw_olist_order_reviews_dataset = spark.read.csv("/obligatorio/data/raw/olist_order_reviews_dataset.csv", header=True, inferSchema=True)
# 6
raw_olist_orders_dataset = spark.read.csv("/obligatorio/data/raw/olist_orders_dataset.csv", header=True, inferSchema=True)
# 7
raw_olist_products_dataset = spark.read.csv("/obligatorio/data/raw/olist_products_dataset.csv", header=True, inferSchema=True)
# 8
raw_olist_sellers_dataset = spark.read.csv("/obligatorio/data/raw/olist_sellers_dataset.csv", header=True, inferSchema=True)
# 9
raw_product_category_name_translation_dataset = spark.read.csv("/obligatorio/data/raw/product_category_name_translation.csv", header=True, inferSchema=True)

Luego se deberá realizar un análisis exploratorio de los datos identificando 
-el tipo de datos que hay en cada columna y que significado tienen dentro del dominio de los datos. 
-- Dentro de un Jupyter notebook se mostrará,
una vista previa de las primeras filas, cantidad de columnas de cada tabla, nombre de cada columna, descripción de
los datos de cada tabla, cómo está compuesto el esquema de los datos, revisar valores nulos o faltantes y limpiarlos
si es necesario. Revisar registros duplicados. Claves primarias únicas. Estos nuevos archivos se guardarán en el
hdfs, en otra ubicación como la versión refinada de los datos. Para todo se debe utilizar únicamente spark.

In [79]:
def analisis_exploratorio(_raw_data, _columns_to_exlude = None):
    print("### Vista inicial de los datos ###")
    _raw_data.show(10)
    print("")
    print("##################################")
    columns = _raw_data.columns
    print(f"Nombres de las columnas: {columns}")
    
    num_columns = len(_raw_data.columns)
    num_rows = _raw_data.count()
    print(f"Cantidad de columnas: {num_columns}")
    print(f"Cantidad de filas: {num_rows}")
    print("")
    print("##################################")
    print("### Schema ###")
    _raw_data.printSchema()
    print("")
    print("##################################")
    print("### Valores null por columnas ###")
    null_counts = []
    for column_name in _raw_data.columns:
        data_type = _raw_data.schema[column_name].dataType.typeName()
        condition = col(column_name).isNull()
        if data_type in ("float", "double"):
            condition = condition | isnan(col(column_name))
        null_counts.append(
            count(when(condition, column_name)).alias(column_name)
        )
    null_counts = _raw_data.select(null_counts)
    null_counts.show()
    
    print("### Elimino valores null ###")
    if _columns_to_exlude:
        columns_to_check = [c for c in _raw_data.columns if c not in _columns_to_exlude]
        _raw_data = _raw_data.dropna(subset=columns_to_check)
    else:
        _raw_data = _raw_data.dropna()
        
    print("")
    print("##################################")
    print("### Valores duplicados por columnas ###")
    duplicates_count = (
        _raw_data.groupBy(columns)
        .agg(count("*").alias("count"))
        .filter("count > 1")
    )
    duplicates_count.show()
    
    total_duplicates = duplicates_count.agg({"count": "sum"}).collect()[0][0]
    print(f"Total de registros duplicados: {total_duplicates if total_duplicates else 0}")
    print("")
    print("### Elimino valores duplicados ###")
    _raw_data = _raw_data.dropDuplicates(columns)
    print("")
    print("##################################")
    print("### Unicidad de claves primarias ###")
    _raw_data.groupBy(columns[0]).count().filter("count > 1").show()

    print("")
    print("##################################")
    print("### Cantidad de columnas curadas y listas para usar ###")
    num_rows = _raw_data.count()
    print(f"Cantidad de filas: {num_rows}")
    
    return _raw_data

In [50]:
########################################################################################################################################

Customer Datasets

In [51]:
cleaned_olist_customers_dataset = analisis_exploratorio(raw_olist_customers_dataset)

### Vista inicial de los datos ###
+--------------------+--------------------+------------------------+--------------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix|       customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|                   14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|                    9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|                    1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|                    8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|                   13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|                   89254|      jaragua do sul|            SC|
|fd826e7cf63160e53..

In [52]:
"""
Donde podemos ver:

customer_id - ID del cliente
customer_unique_id - ID unico que identifica al cliente en otros datasets
customer_zip_code_prefix - Codigo prefijo del zip del cliente
customer_city - Ciudad
customer_state - Estado del cliente
"""

'\nDonde podemos ver:\n\ncustomer_id - ID del cliente\ncustomer_unique_id - ID unico que identifica al cliente en otros datasets\ncustomer_zip_code_prefix - Codigo prefijo del zip del cliente\ncustomer_city - Ciudad\ncustomer_state - Estado del cliente\n'

In [53]:
########################################################################################################################################

Geolocalización Datasets

In [54]:
cleaned_olist_geolocation_dataset = analisis_exploratorio(raw_olist_geolocation_dataset)

### Vista inicial de los datos ###
+---------------------------+-------------------+------------------+----------------+-----------------+
|geolocation_zip_code_prefix|    geolocation_lat|   geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+-------------------+------------------+----------------+-----------------+
|                       1037| -23.54562128115268|-46.63929204800168|       sao paulo|               SP|
|                       1046|-23.546081127035535|-46.64482029837157|       sao paulo|               SP|
|                       1046| -23.54612896641469|-46.64295148361138|       sao paulo|               SP|
|                       1041|  -23.5443921648681|-46.63949930627844|       sao paulo|               SP|
|                       1035|-23.541577961711493|-46.64160722329613|       sao paulo|               SP|
|                       1012|-23.547762303364266|-46.63536053788448|       são paulo|               SP|
|                       1047|

+---------------------------+---------------+---------------+----------------+-----------------+
|geolocation_zip_code_prefix|geolocation_lat|geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+---------------+---------------+----------------+-----------------+
|                          0|              0|              0|               0|                0|
+---------------------------+---------------+---------------+----------------+-----------------+

### Elimino valores null ###

##################################
### Valores duplicados por columnas ###


+---------------------------+-------------------+-------------------+----------------+-----------------+-----+
|geolocation_zip_code_prefix|    geolocation_lat|    geolocation_lng|geolocation_city|geolocation_state|count|
+---------------------------+-------------------+-------------------+----------------+-----------------+-----+
|                       1026| -23.53925652899767|-46.633440525535235|       sao paulo|               SP|    2|
|                       1106|-23.531062064948905|-46.629505278925556|       sao paulo|               SP|    2|
|                       1241|-23.545792634196992|  -46.6567234730798|       sao paulo|               SP|    3|
|                       1226|-23.538190850683794|-46.651323227306854|       são paulo|               SP|    4|
|                       1223|-23.543735510964265| -46.65415427699233|       sao paulo|               SP|    2|
|                       1303|-23.551504721622702| -46.64945164585375|       sao paulo|               SP|    3|
|

Total de registros duplicados: 390005

### Elimino valores duplicados ###

##################################
### Unicidad de claves primarias ###


+---------------------------+-----+
|geolocation_zip_code_prefix|count|
+---------------------------+-----+
|                       6654|  128|
|                       7240|  138|
|                      12940|  187|
|                      13289|   48|
|                      13285|   42|
|                      13840|   97|
|                      18800|  239|
|                      20735|  160|
|                      26087|   99|
|                      28024|   84|
|                       1238|   72|
|                       2366|   18|
|                      18051|  154|
|                      21220|  176|
|                       6397|   81|
|                       6620|   50|
|                       2866|   31|
|                       4101|   28|
|                       3175|   27|
|                       5518|   25|
+---------------------------+-----+
only showing top 20 rows



In [55]:
"""
Las claves de zipcode se encuentran duplicadas, realizando el analisis de variabilidad tenemos que corresponden a distintas geolocalizaciones. 
Por esto, no es que esté mal la duplicación, sino que están asociados los codigos a distintas ubicaciones geográficas como se puede ver en geolocation_city. 
"""

'\nLas claves de zipcode se encuentran duplicadas, realizando el analisis de variabilidad tenemos que corresponden a distintas geolocalizaciones. \nPor esto, no es que esté mal la duplicación, sino que están asociados los codigos a distintas ubicaciones geográficas como se puede ver en geolocation_city. \n'

In [56]:
cleaned_olist_geolocation_dataset.groupBy("geolocation_zip_code_prefix", "geolocation_city").count().show()

+---------------------------+--------------------+-----+
|geolocation_zip_code_prefix|    geolocation_city|count|
+---------------------------+--------------------+-----+
|                       9181|         santo andre|   77|
|                       9134|         santo andre|   25|
|                       9407|      ribeirao pires|   20|
|                      23016|      rio de janeiro|   48|
|                       6145|              osasco|   25|
|                      12093|             taubate|   17|
|                      20251|      rio de janeiro|  126|
|                      22050|      rio de janeiro|  107|
|                       3141|           sao paulo|   19|
|                       6783|     taboao da serra|   83|
|                      12249|são francisco xavier|    1|
|                      15850|              urupês|   12|
|                       1324|           são paulo|    8|
|                       2169|           sao paulo|   27|
|                       2804|  

In [57]:
"""
Donde podemos ver:

Nombres de las columnas: ['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state']

geolocation_zip_code_prefix - Codigo referente al zip
geolocation_lat - Coordenada de latitud
geolocation_lng - Coordenada de longitud
geolocation_city - Ciudad
geolocation_state - Estado
"""

"\nDonde podemos ver:\n\nNombres de las columnas: ['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state']\n\ngeolocation_zip_code_prefix - Codigo referente al zip\ngeolocation_lat - Coordenada de latitud\ngeolocation_lng - Coordenada de longitud\ngeolocation_city - Ciudad\ngeolocation_state - Estado\n"

Order Items Dataset

In [58]:
cleaned_olist_order_items_dataset = analisis_exploratorio(raw_olist_order_items_dataset)

### Vista inicial de los datos ###
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|0004

+--------+-------------+----------+---------+-------------------+-----+-------------+-----+
|order_id|order_item_id|product_id|seller_id|shipping_limit_date|price|freight_value|count|
+--------+-------------+----------+---------+-------------------+-----+-------------+-----+
+--------+-------------+----------+---------+-------------------+-----+-------------+-----+



Total de registros duplicados: 0

### Elimino valores duplicados ###

##################################
### Unicidad de claves primarias ###


+--------------------+-----+
|            order_id|count|
+--------------------+-----+
|7a70b827ebc6ab85b...|    3|
|873c916c2bff3ea61...|    2|
|3e22c141d28618608...|    2|
|5e57ff5e1c008db89...|    2|
|0cb32545f515420d9...|    2|
|99986880076491bbf...|    2|
|118045506e1c1dda0...|    5|
|2f5e702c70321e3f1...|    2|
|5137d2621656ff029...|    2|
|125e0c61fdbfc589f...|    4|
|014405982914c2cde...|    2|
|116debdd5705aaa51...|    2|
|0baa56401ae212e30...|    2|
|51a5a1d89c87180ca...|    2|
|05afef1c185862cab...|    2|
|078c2d9bb7e5905e8...|    2|
|6386b8aabb7f032b7...|    2|
|829f21ada22c20b2a...|    2|
|7de7f006c92939fa5...|    2|
|1c4a92d82c1b0dec1...|    3|
+--------------------+-----+
only showing top 20 rows



In [ ]:
"""
Podemos verificar que existen ordenes de items repetidas. Lo cual deberíamos verificar estas en que se diferencian.
Filtramos uno random para obtener el valor de la orden e imprimimos el valor de las ordenes
"""

In [68]:
order_duplicated_df = cleaned_olist_order_items_dataset.groupBy('order_id').count().filter("count > 1")
id_order_duplicated = order_duplicated_df.head(1)[0]['order_id']

filtered_df = cleaned_olist_order_items_dataset.filter(cleaned_olist_order_items_dataset['order_id'] == id_order_duplicated)
filtered_df.show()

+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+
|7a70b827ebc6ab85b...|            2|2c3f424b5643b0366...|888faa8bfb0b159c3...|2018-08-13 17:50:09|335.9|         2.24|
|7a70b827ebc6ab85b...|            3|39b146e69c32ffe0a...|0b35c634521043bf4...|2018-08-15 17:50:09| 99.9|         56.1|
|7a70b827ebc6ab85b...|            1|39b146e69c32ffe0a...|0b35c634521043bf4...|2018-08-15 17:50:09| 99.9|         56.1|
+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+



In [ ]:
"""
Concluimos que es normal debido a que los valores de las order_item_id son distintos. 
"""

In [69]:
"""
Donde podemos ver:

['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']

order_id - ID de orden
order_item_id - ID de la orden del item
seller_id - ID del vendedor
customer_city - Ciudad del cliente
customer_state - Estado del cliente
"""

"\nDonde podemos ver:\n\n['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']\n\norder_id - ID de orden\norder_item_id - ID de la orden del item\nseller_id - ID del vendedor\ncustomer_city - Ciudad del cliente\ncustomer_state - Estado del cliente\n"

Order Payments Dataset

In [71]:
cleaned_olist_order_payments_dataset = analisis_exploratorio(raw_olist_order_payments_dataset)

### Vista inicial de los datos ###
+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|b81ef226f3fe1789b...|                 1| credit_card|                   8|        99.33|
|a9810da82917af2d9...|                 1| credit_card|                   1|        24.39|
|25e8ea4e93396b6fa...|                 1| credit_card|                   1|        65.71|
|ba78997921bbcdc13...|                 1| credit_card|                   8|       107.78|
|42fdf880ba16b47b5...|                 1| credit_card|                   2|       128.45|
|298fcdf1f73eb413e...|                 1| credit_card|                   2|        96.12|
|771ee386b001f0620...|                 1| credit_card|                   1|        81.16|
|3d7239c394a212faa...|                 1| credit_card|           

In [73]:
order_order_payments_df = cleaned_olist_order_payments_dataset.groupBy('order_id').count().filter("count > 1")
id_order_payments_duplicated = order_order_payments_df.head(1)[0]['order_id']

filtered_df = cleaned_olist_order_payments_dataset.filter(cleaned_olist_order_payments_dataset['order_id'] == id_order_payments_duplicated)
filtered_df.show()

+--------------------+------------------+------------+--------------------+-------------+
|            order_id|payment_sequential|payment_type|payment_installments|payment_value|
+--------------------+------------------+------------+--------------------+-------------+
|f44cb69655f8e4d13...|                 1| credit_card|                   5|       146.32|
|f44cb69655f8e4d13...|                 2|     voucher|                   1|         18.0|
+--------------------+------------------+------------+--------------------+-------------+



In [74]:
"""
Donde podemos ver:

nota: Similar al caso anterior, las secuencias son distintas de pago y el payment type tambien. Es correcto estos registros.

Similar al anterior, 
order_id - ID del Orden
payment_sequential - 
payment_type - Tipo de pago
payment_installments - 
payment_value - Valor del pago
"""

'\nDonde podemos ver:\n\nnota: Similar al caso anterior, las secuencias son distintas de pago y el payment type tambien. Es correcto estos registros.\n\nSimilar al anterior, \norder_id - ID del Orden\npayment_sequential - \npayment_type - Tipo de pago\npayment_installments - \npayment_value - Valor del pago\n'

Order Reviews Dataset

In [80]:
 cleaned_olist_order_reviews_dataset = analisis_exploratorio(raw_olist_order_reviews_dataset)

### Vista inicial de los datos ###
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                null|                  null| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                null|                  null| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                null|                  null| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|                null|  Recebi bem

+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+-----+
|review_id|order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|count|
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+-----+
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+-----+

Total de registros duplicados: 0

### Elimino valores duplicados ###

##################################
### Unicidad de claves primarias ###
+--------------------+-----+
|           review_id|count|
+--------------------+-----+
|f45e3373e177f5694...|    2|
|fbb7c6f69326ad5bf...|    2|
|70be3f25177bd0d08...|    2|
|7168439a7228c5684...|    2|
|23a008b7ae9704e3b...|    2|
|58f1655df206a9a40...|    2|
|471a7d853b152d77a...|    2|
|56647f8fbe7430f42...|    2|
|74090c1156e291923...|

In [81]:
"""
Hecha la limpieza, vemos que se reduce considerablemente el nro total, pero los valores son de columnas las cuales no deberían afectar y están bien que sean nulls.
Columnas como : review_comment_title|review_comment_message
Son validas nulls. Por lo cual debemos contemplar y exluir ese valor.
"""

'\nHecha la limpieza, vemos que se reduce considerablemente el nro total, pero los valores son de columnas las cuales no deberían afectar y están bien que sean nulls.\nColumnas como : review_comment_title|review_comment_message\nSon validas nulls. Por lo cual debemos contemplar y exluir ese valor.\n'

In [82]:
 cleaned_olist_order_reviews_dataset = analisis_exploratorio(raw_olist_order_reviews_dataset, ["review_comment_title","review_comment_message"])

### Vista inicial de los datos ###
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                null|                  null| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                null|                  null| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                null|                  null| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|                null|  Recebi bem

+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+-----+
|review_id|order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|count|
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+-----+
+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+-----+



Total de registros duplicados: 0

### Elimino valores duplicados ###

##################################
### Unicidad de claves primarias ###


+--------------------+-----+
|           review_id|count|
+--------------------+-----+
|a8ffb30c74397c7cf...|    2|
|f144ac19984746532...|    2|
|e4f5fbbbf2fa8f259...|    2|
|c9a406d3076559e35...|    2|
|dd43a798c9e8baf7b...|    2|
|9a07cb8136a1792e7...|    2|
|dbcd301b59c7e85e6...|    2|
|ef60daa08e71f279b...|    2|
|4e9a141db068cf685...|    2|
|4812973f7fb59a488...|    2|
|9d7b240dff5539983...|    2|
|c1dd7c6a2154caaf1...|    2|
|bee3c76b6017ec9de...|    2|
|77d427beb34a26b48...|    2|
|f3544b47fdfb4bdd2...|    2|
|59d160f8ce9217282...|    2|
|89d6214895235bb95...|    2|
|090aa2980b0aea78c...|    2|
|9840563f4c2189d0a...|    2|
|8b8f495d30fca09fd...|    2|
+--------------------+-----+
only showing top 20 rows


##################################
### Cantidad de columnas curadas y listas para usar ###


Cantidad de filas: 95377


In [83]:
"""
Donde podemos ver:

['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp']

review_id - Id de la reseña
order_id - ID de la orden
review_score - Valor de la reseña
review_comment_title - Titulo del comentario de la reseña
review_comment_message - Comentario asociado a la reseña
review_creation_date - Fecha de creación de la reseña
review_answer_timestamp - Fecha de respuesta de la reseña

"""

"\nDonde podemos ver:\n\n['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp']\n\nreview_id - Id de la reseña\norder_id - ID de la orden\nreview_score - Valor de la reseña\nreview_comment_title - Titulo del comentario de la reseña\nreview_comment_message - Comentario asociado a la reseña\nreview_creation_date - Fecha de creación de la reseña\nreview_answer_timestamp - Fecha de respuesta de la reseña\n\n"

Orders Dataset 

In [84]:
 cleaned_olist_orders_dataset = analisis_exploratorio(raw_olist_orders_dataset)

### Vista inicial de los datos ###
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb91

+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+-----+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|count|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+-----+
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+-----+



Total de registros duplicados: 0

### Elimino valores duplicados ###

##################################
### Unicidad de claves primarias ###


+--------+-----+
|order_id|count|
+--------+-----+
+--------+-----+


##################################
### Cantidad de columnas curadas y listas para usar ###


Cantidad de filas: 96461


In [85]:
"""
Donde podemos ver:

order_id - Id de la orden
customer_id - ID del cliente
order_status - Estado de orden
order_purchase_timestamp - Fecha que ocurrió la orden de compra
order_approved_at - Fecha de aprovación de la orden
order_delivered_carrier_date - Fecha de currier para entrega
order_delivered_customer_date - Fecha de entrega de la orden al cliente
order_estimated_delivery_date - Fecha de entrega estimada al cliente
""" 

'\nDonde podemos ver:\n\norder_id - Id de la orden\ncustomer_id - ID del cliente\norder_status - Estado de orden\norder_purchase_timestamp - Fecha que ocurrió la orden de compra\norder_approved_at - Fecha de aprovación de la orden\norder_delivered_carrier_date - Fecha de currier para entrega\norder_delivered_customer_date - Fecha de entrega de la orden al cliente\norder_estimated_delivery_date - Fecha de entrega estimada al cliente\n'

Olist Products Dataset

In [86]:
  cleaned_olist_products_dataset = analisis_exploratorio(raw_olist_products_dataset)

### Vista inicial de los datos ###
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                

In [87]:
"""
Donde podemos ver:

product_id - ID del producto
product_category_name - Categoria del producto
product_name_lenght - Largo del nombre del producto
product_description_lenght - Descripción del producto
product_photos_qty - Cantidad de fotos
product_weight_g - Dimension del producto - gramos
product_length_cm - Dimension del producto largo - cm
product_height_cm - Dimension del producto altura- cm
product_width_cm - Dimension del producto produndidad- cm

"""

'\nDonde podemos ver:\n\nproduct_id - ID del producto\nproduct_category_name - Categoria del producto\nproduct_name_lenght - Largo del nombre del producto\nproduct_description_lenght - Descripción del producto\nproduct_photos_qty - Cantidad de fotos\nproduct_weight_g - Dimension del producto - gramos\nproduct_length_cm - Dimension del producto largo - cm\nproduct_height_cm - Dimension del producto altura- cm\nproduct_width_cm - Dimension del producto produndidad- cm\n\n'

Sellers Dataset

In [88]:
cleaned_olist_sellers_dataset = analisis_exploratorio(raw_olist_sellers_dataset)

### Vista inicial de los datos ###
+--------------------+----------------------+-----------------+------------+
|           seller_id|seller_zip_code_prefix|      seller_city|seller_state|
+--------------------+----------------------+-----------------+------------+
|3442f8959a84dea7e...|                 13023|         campinas|          SP|
|d1b65fc7debc3361e...|                 13844|       mogi guacu|          SP|
|ce3ad9de960102d06...|                 20031|   rio de janeiro|          RJ|
|c0f3eea2e14555b6f...|                  4195|        sao paulo|          SP|
|51a04a8a6bdcb23de...|                 12914|braganca paulista|          SP|
|c240c4061717ac180...|                 20920|   rio de janeiro|          RJ|
|e49c26c3edfa46d22...|                 55325|           brejao|          PE|
|1b938a7ec6ac5061a...|                 16304|        penapolis|          SP|
|768a86e36ad6aae3d...|                  1529|        sao paulo|          SP|
|ccc4bbb5f32a6ab2b...|                 80

In [ ]:
"""
Donde podemos ver:

seller_id - ID del vendedor
seller_zip_code_prefix - Codigo postal del vendedor
seller_city - Ciudad del vendedor
seller_state - Estado del vendedor
"""

In [ ]:
Products Category Name Traslation

In [89]:
 cleaned_product_category_name_translation = analisis_exploratorio(raw_product_category_name_translation)

### Vista inicial de los datos ###
+---------------------+-----------------------------+
|product_category_name|product_category_name_english|
+---------------------+-----------------------------+
|         beleza_saude|                health_beauty|
| informatica_acess...|         computers_accesso...|
|           automotivo|                         auto|
|      cama_mesa_banho|               bed_bath_table|
|     moveis_decoracao|              furniture_decor|
|        esporte_lazer|               sports_leisure|
|           perfumaria|                    perfumery|
| utilidades_domest...|                   housewares|
|            telefonia|                    telephony|
|   relogios_presentes|                watches_gifts|
+---------------------+-----------------------------+
only showing top 10 rows


##################################
Nombres de las columnas: ['product_category_name', 'product_category_name_english']
Cantidad de columnas: 2
Cantidad de filas: 71

################

In [90]:
"""
Donde podemos ver:

product_category_name - Categoria del producto
product_category_name_english - Categoria del producto en ingles

"""

'\nDonde podemos ver:\n\nproduct_category_name - Categoria del producto\nproduct_category_name_english - Categoria del producto en ingles\n\n'

Escribir resultados en HDSF

In [99]:
directory_results = "obligatorio/data/cleaned"

In [100]:
_path = f"hdfs://localhost:9000/{directory_results}/"
_path

'hdfs://localhost:9000/obligatorio/data/cleaned/'

In [101]:
cleaned_olist_customers_dataset.write.csv(_path+"/cleaned_olist_customers_dataset.csv", header=True, mode="overwrite")

In [102]:
cleaned_olist_geolocation_dataset.write.csv(_path+"/cleaned_olist_geolocation_dataset.csv", header=True, mode="overwrite")

In [103]:
cleaned_olist_order_items_dataset.write.csv(_path+"/cleaned_olist_order_items_dataset.csv", header=True, mode="overwrite")

In [104]:
cleaned_olist_order_payments_dataset.write.csv(_path+"/cleaned_olist_order_payments_dataset.csv", header=True, mode="overwrite")

In [105]:
cleaned_olist_order_reviews_dataset.write.csv(_path+"/cleaned_olist_order_reviews_dataset.csv", header=True, mode="overwrite")

In [106]:
cleaned_olist_orders_dataset.write.csv(_path+"/cleaned_olist_orders_dataset.csv", header=True, mode="overwrite")

In [107]:
cleaned_olist_products_dataset.write.csv(_path+"/cleaned_olist_products_dataset.csv", header=True, mode="overwrite")

In [108]:
cleaned_olist_sellers_dataset.write.csv(_path+"/olist_sellers_dataset.csv", header=True, mode="overwrite")

In [110]:
cleaned_product_category_name_translation.write.csv(_path+"/cleaned_product_category_name_translation.csv", header=True, mode="overwrite")